Multi-armed bandits in probability theory are a set of problems that essentially aim to optimize long term returns when given K potential options. Specifically the name is a good adage to summarize this problem, given a gambler facing a row of multiple single arm bandits (slot machines, pokies) what is the best playing strategy to optimize returns? In other words, given a finite set of resources (the gamblers money) that will be spent among competing options, which at the time of playing may only be partially known and may be better determined as further rounds are played what is the best way to maximize his returns. This leads into the idea of exploration vs exploitation, should the gambler keep playing one machine until it pays, should he distribute his money evenly among the bandits and hope one pays off, or is there a better strategy that will help maximize his returns? We can assume that if one machine is consistently paying the gambler should be allocating his money to it, but what if there are other machines that could be potentially paying off the same or if not more that he is not focused on? The take away here is how do we determine the optimal way to explore and exploit to maximize returns.

Let's take this example, the gambler starts by even distributing his money among K machines. When enough rounds are played and there is a machine that performs better than the rest the gambler takes portions from other machines and dynamically allocates it to this one, whilst periodically checking to see if the other machines are becoming more promising. Here the gambler is continuing to exploit the paying machine whilst continuing to explore for other bandits that will potentially offer returns as well.

We can implement some code to demonstrate this exact scenario, but first let's dig a little bit into the theory of MAB and Upper Confidence Bound solution to the multi-armed bandit problem and how it solves the exloration vs exploitation dilemma.

A multi-armed bandit can be thought of as a tuple $\langle\mathcal{A}, \mathcal{R}\rangle$. 

Where by $\mathcal{A}$ in our example can be seen as a set of actions or interactions ie. with our bandits. At each step t the agent selects an action $a_{t} \in \mathcal{A}$ . 

$\mathcal{R}^{a}(r)=\mathbb{P}[r | a]$ is an unknown probability distribution of potential rewards, which we can think of as our expected payoffs. In our interactions, the bandit that gets played will generate some reward $r_{t} \sim \mathcal{R}^{a_{t}}$ our aim in this process is to maximise cumulative reward $\sum_{\tau=1}^{t} r_{\tau}$ - pick as many winners as possible.

The action-value is the mean reward for action a, $Q(a)=\mathbb{E}[r | a]$.

This process can be viewed a simplified version of the Markov decision process, given the absence of state.

Now how do we determine the best course of action? We know that we want to pick the most profitable machine, but if we were to focus on one, how would we account for the potential losses from ignoring equally or more profitable machines? This is where a concept called regret comes into the picture. Our aim in this exercise is not only to maximize reward (profits) but also minimize regret (lost optimal opportunities).

The optimal value is described as $V^{*}$ is $V^{*}=Q\left(a^{*}\right)=\max _{a \in \mathcal{A}} Q(a)$

The incured regret for one time step is $I_{t}=\mathbb{E}\left[V^{*}-Q\left(a_{t}\right)\right]$

The total regret for the total opportunity loss $L_{t}=\mathbb{E}\left[\sum_{\tau=1}^{t} V^{*}-Q\left(a_{\tau}\right)\right]$

Therefore our optimal way to get the best return is to maximise cumulative reward and to minimise total regret.

### Optimism in the face of uncertainty

In UCB the more uncertain we are about a given action the more important it is to explore it as it could turn out to be the best action.

UCB measures the potenital for a given action by estimating the upper confidence $\hat{U}_t(a)$ for each action value, such that $Q(a) \leq \hat{Q}_t(a) + \hat{U}_t(a)$.

$\hat{U}_t(a)$ is a function of ${N_{t}(a)}$, and with a small ${N_{t}(a)}$ we have a large $\hat{U}_t(a)$ and thus we have a high uncertainty of our estimation. Conversely if we have a large ${N_{t}(a)}$ we have a small $\hat{U}_t(a)$ and our estimate is more accurate.

UCB takes a gready approach and will always select the action that maximizes UCB: $a_t = argmax_{a \in \mathcal{A}} \hat{Q}_t(a) + \hat{U}_t(a)$

So now we must estimate UCB and to this we need to introduce Hoeffding’s Inequality.

Hoeffding’s Inequality is an important technique in learning theory. H.I basically provides an upper bound on the probability that the sample mean of some distribution deviates from the true mean by more that a certain amount. 

In our application, let's assume that $X_1, \dots, X_t$ are independent and identically distributed random variables bound between 0 and 1.

The sample mean is: $\overline{X}_t = \frac{1}{t}\sum_{\tau=1}^t X_\tau$ 

Then we have: $\mathbb{P} [ \mathbb{E}[X] > \overline{X}_t + u] \leq e^{-2tu^2}$

Now, if we apply this to our problem, for a given acion a we will have: $\mathbb{P} [ Q(a) > \hat{Q}_t(a) + U_t(a)] \leq e^{-2t{U_t(a)}^2}$

Where:
-  $Q(a)$ is our true mean.
-  $\hat{Q}_t(a)$ is our sample mean.
-  $U_t(a)$ is our upper confidence bound.

So we are basically making the best guess we can, given that each of our $a$'s will have different rewards we can use this inequality to pick a bound so the probability that the true value will exceed UCB.

Now we can solve for $U_t(a)$ by setting $e^{-2t U_t(a)^2} = p$ and solving: $U_t(a) = \sqrt{\frac{-\log p}{2 N_t(a)}}$

Now as we observe the rewards as we make a number of actions we can begin to reduce $p$, if we set $p = t^{-4}$ we get $U_t(a) = \sqrt{\frac{2 \log t}{N_t(a)}}$ and this ensures we select optimal action as t → ∞.

We arrive at our final formula:

$A_{t}=\underset{a \in \mathcal{A}}{\operatorname{argmax}} Q(a)+\sqrt{\frac{2 \log t}{N_{t}(a)}}$

### Example of UCB in Action

To demonstrate the theory that we just worked through we have a simple example written below. The context for this example is exactly the MAB problem where we have n_bandits (number of slot machines) and money (assume $1 per play) which limits the number of rounds we play.

In this example the user is asked to input a reward, 0 or 1 to simulate a successful encounter with a given bandit that has been determined by the UCB algortithm. 

In [1]:
import math

class UCB1:
    """Class for running a toy example of MAB UCB1 problem.
    
    :param money: How much money the gambler has to spend.
    :type money: int
    :param n_bandits: How many bandits there are to play.
    :type n_bandits: int
    """
    def __init__(self, money, n_bandits):
        self.money = money
        self.n_bandits = n_bandits
        self.bandits_played = []
        self.bandits_returns = {x:0 for x in range(self.n_bandits)}
        self.n_selections = {x:0 for x in range(self.n_bandits)}
        self.total_return = 0
        self.bounds = {x:[] for x in range(self.n_bandits)}
        
    def calc_ucb(self, i, t):
        if self.n_selections[i] > 0:
            Qa = self.bandits_returns[i] / self.n_selections[i]
            Ut = math.sqrt(2 * math.log(t + 1) / self.n_selections[i])
            ub = Qa + Ut
            print('Upper bound: ', ub)
            self.bounds[i].append(ub)
        else:
            ub = 10
            self.bounds[i].append(ub)
        return {i:ub}
        
    def play_round(self, t):
        res = {k: v for d in [self.calc_ucb(x, t) for x in range(self.n_bandits)] for k, v in d.items()}
        bandit = max(res, key=lambda v: res[v])
        self.bandits_played.append(bandit)
        print('Bandit: ', bandit)
        self.n_selections[bandit] += 1
        cash_return = int(input())
        self.bandits_returns[bandit] += cash_return
        self.total_return += cash_return
        
    def run(self):
        [self.play_round(x) for x in range(self.money)]
        print(f'Bandit {max(self.bandits_returns, key=lambda v: self.bandits_returns[v])} was the optimal machine.')

In [2]:
ucb = UCB1(10, 3)

In [3]:
ucb.run()

Bandit:  0
1
Upper bound:  2.177410022515475
Bandit:  1
0
Upper bound:  2.4823038073675114
Upper bound:  1.4823038073675112
Bandit:  2
0
Upper bound:  2.6651092223153956
Upper bound:  1.6651092223153954
Upper bound:  1.6651092223153954
Bandit:  0
1
Upper bound:  2.26863624117952
Upper bound:  1.7941225779941015
Upper bound:  1.7941225779941015
Bandit:  0
1
Upper bound:  2.0929347248663586
Upper bound:  1.8930184728248454
Upper bound:  1.8930184728248454
Bandit:  0
1
Upper bound:  1.9863848511243756
Upper bound:  1.9727697022487511
Upper bound:  1.9727697022487511
Bandit:  0
1
Upper bound:  1.9120178817720266
Upper bound:  2.039333980337618
Upper bound:  2.039333980337618
Bandit:  1
0
Upper bound:  1.9374912431241627
Upper bound:  1.4823038073675112
Upper bound:  2.09629414793641
Bandit:  2
0
Upper bound:  1.9597051824376162
Upper bound:  1.5174271293851465
Upper bound:  1.5174271293851465
Bandit:  0
1
Bandit 0 was the optimal machine.


In this example we see that bandit 0 was favoured for reward, although this example has a very limited $t$, we see that our algorithm picks up on the fact that bandit 0 has the best yield. We can also see that there is continued exploration of the other bandits.

As another demonstration, we will observe what happens if there are multiple high yeilding bandits.

In [4]:
ucb2 = UCB1(10, 3)
ucb2.run()

Bandit:  0
1
Upper bound:  2.177410022515475
Bandit:  1
1
Upper bound:  2.4823038073675114
Upper bound:  2.4823038073675114
Bandit:  2
0
Upper bound:  2.6651092223153956
Upper bound:  2.6651092223153956
Upper bound:  1.6651092223153954
Bandit:  0
1
Upper bound:  2.26863624117952
Upper bound:  2.7941225779941012
Upper bound:  1.7941225779941015
Bandit:  1
1
Upper bound:  2.33856619904585
Upper bound:  2.33856619904585
Upper bound:  1.8930184728248454
Bandit:  0
1
Upper bound:  2.1389791186424545
Upper bound:  2.3949588341794583
Upper bound:  1.9727697022487511
Bandit:  1
1
Upper bound:  2.177410022515475
Upper bound:  2.177410022515475
Upper bound:  2.039333980337618
Bandit:  0
1
Upper bound:  2.048147073968205
Upper bound:  2.2102959906117237
Upper bound:  2.09629414793641
Bandit:  1
1
Upper bound:  2.0729830131446736
Upper bound:  2.0729830131446736
Upper bound:  2.145966026289347
Bandit:  2
0
Bandit 0 was the optimal machine.


We see here that both 0 and 1 are superior, so the algorthim favors these, but still visits bandit 2 for continued exploration.

In [5]:
ucb2.bandits_returns

{0: 4, 1: 4, 2: 0}